In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "filmy"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
seznam_filmu = seznam_souboru(slozka)
seznam_filmu[0:10]

['10144-tabor-padlych-zen.html',
 '10173-eastern-krvavy-hugo.html',
 '102282-o-jecminkovi.html',
 '102314-zimni-vila.html',
 '10242-narkoblues.html',
 '103341-jan-hus-mse-za-tri-mrtve-muze.html',
 '103354-obcansky-prukaz.html',
 '103358-7-dni-hrichu.html',
 '103360-vojta-lavicka-nahoru-a-dolu.html',
 '103369-kuky-se-vraci.html']

## Začínáme scrapovat

### Nejdřív zjistíme, co všechno budeme scrapovat

In [5]:
kolik = None

(Čím menší číslo, tím menší vzorek a tím rychlejší test. Pro zpracování kompletního seznamu nutno změnit proměnou na None.)

In [6]:
def jake_jsou_sekce(slozka):
    sekce = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-section')
            for item_detail in item_details:
                h2_tags = item_detail.find_all('h2')
                for h2_tag in h2_tags:
                    if h2_tag.get_text() not in sekce:
                        sekce.append(h2_tag.get_text())
    return(sekce)

In [7]:
sekce = jake_jsou_sekce("filmy")

In [8]:
sekce

['Anotace',
 'Obsah',
 'Hrají',
 'Štáb a tvůrci',
 'Hudba',
 'Lokace',
 'Produkční údaje',
 'Technické údaje',
 'Knihovna',
 'Poznámka',
 'Ocenění',
 'Galerie',
 'Komentář',
 'Písně',
 'Revue',
 'Účinkují',
 'Mluví',
 'Film online']

In [9]:
def jake_jsou_detaily(slozka):
    detaily = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='item-details')
            for item_detail in item_details:
                h3_tags = item_detail.find_all('h3')
                for h3_tag in h3_tags:
                    if h3_tag.get_text() not in detaily:
                        detaily.append(h3_tag.get_text())
    return(detaily)

In [10]:
detaily = jake_jsou_detaily(slozka)

In [11]:
def jake_jsou_kredity(slozka):
    kredity = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Štáb a tvůrci" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in kredity:
                            kredity.append(h3_tag.get_text())
    return(kredity)

In [12]:
kredity = jake_jsou_kredity(slozka)

In [13]:
def jake_jsou_technicke_detaily(slozka):
    technika = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Technické údaje" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in technika:
                            technika.append(h3_tag.get_text())
    return(technika)

In [14]:
technika = jake_jsou_technicke_detaily(slozka)

In [15]:
technika

['Délka',
 'Minutáž',
 'Distribuční nosič',
 'Poměr stran',
 'Barva',
 'Zvuk',
 'Zvukový systém/formát',
 'Verze',
 'Mluveno',
 'Podtitulky',
 'Úvodní/závěrečné titulky',
 'Původní metráž',
 'Mezititulky',
 'Animační technika',
 'Natáčecí formát',
 'Rychlost']

## Scrapování filmů

In [16]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [17]:
def pridej_atributy(slovnik, seznam, nalezene):
    for s in seznam:
        for n in nalezene:
            if s in n:
                n = n.replace(s," ")
                slovnik[s] = n.strip()

In [27]:
def scrape_film(slozka, soubor):

    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    atributy = {}
        
    atributy["Film"] = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""

    ## Detaily – země původu, copyright…
    
    item_details = soup.find_all(class_='item-details')
    divs_text = []

    for item_detail in item_details:
        divs = item_detail.find_all('div')
        for div in divs:
            obsah = div.text
            if div.find("span"):
                obsah = div.find("h3").text + ", ".join([s.text for s in div.find_all(class_="comma")])
            divs_text.append(obsah)

    pridej_atributy(atributy, detaily, divs_text)
    
    ## Credits
    
    item_details = soup.find_all(class_='db-film-credits-section')
    divs_text = []
    mluvi = []

    for item_detail in item_details:
        
        if "Hrají" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_="credit-item")
            hraji = []
            postavy = []
            
            for div in divs:
                if div.find("a"):
                    try:
                        jmeno = f"""{div.find("h3").text} ({id_persony(div.find("h3").find("a")["href"])})"""
                    except:
                        jmeno = f"""{div.find("h3").text}"""
                    hraji.append(jmeno)
                if div.find("p"):
                    role = div.find("p").text
                    if div.find("p").find("a"):
                        role = role.replace(div.find("p").find("a").text,"")
                        mluvi.append(f"""{div.find("a").text} ({id_persony(div.find("a")["href"])})""")
                    if div.find("p").find("span"):
                        role = role.replace(div.find("p").find("span").text,"")
                    postavy.append(role.strip())
            atributy["Hrají"] = hraji
            atributy["Postavy"] = postavy
            
        if "Mluví" in item_detail.find("h2").text:
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                if div.find("a"):
                    jmeno = f"""{div.find("h3").text} ({id_persony(div.find("a")["href"])})"""
                    mluvi.append(jmeno)
            
        if mluvi:
            atributy["Mluví"] = mluvi
            
        if "Komentář" in item_detail.find("h2").text:
            komentar = []
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                if div.find("a"):
                    jmeno = f"""{div.find("h3").text} ({id_persony(div.find("a")["href"])})"""
                    komentar.append(jmeno)
            atributy["Komentář"] = komentar

        if "Účinkují" in item_detail.find("h2").text:
            ucinkuji = []
            divs = item_detail.find_all(class_="credit-item")
            for div in divs:
                if div.find("a"):
                    jmeno = f"""{div.find("h3").text} ({id_persony(div.find("a")["href"])})"""
                    ucinkuji.append(jmeno)
            atributy["Účinkují"] = ucinkuji
            
        if "Štáb a tvůrci" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find_all("a"):
                    prvni = div.find("h3").text
                    odkazy = div.find_all("a")
                    odkazy2 = []
                    for a in odkazy:
                        id = id_persony(a["href"])
                        odkazy2.append(f"{a.text} ({id})")
                        druhy = ", ".join(odkazy2)
                        divs_text.append(prvni + " " + druhy)
                else:
                    divs_text.append(div.text.strip())
            
            pridej_atributy(atributy, kredity, divs_text)
            
        if "Technické údaje" in item_detail.find("h2").text or "Produkční údaje" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find("h3"):
                    if div.find("p"):
                        atributy[div.find("h3").text.strip()] = div.find("p").text.strip()
                        if "Žánr" in div.find("h3").text:
                            zanry = [z.strip() for z in div.find("p").text.split(",")]
                            zanry = ", ".join(zanry)
                            atributy[div.find("h3").text.strip()] = zanry
                        if div.find(class_="comma"):
                            atributy[div.find("h3").text.strip()] = ", ".join([s.text.strip() for s in div.find_all(class_="comma")])
                        
        if "Písně" in item_detail.find("h2").text:
            pisne = item_detail.find_all('h3')
            pisne = [pisen.text.strip() for pisen in pisne]
            atributy["Písně"] = pisne
            
        if "Lokace" in item_detail.find("h2").text:
            lokace = item_detail.find('p')
            atributy["Lokace"] = lokace.text.strip()

        if "Ocenění" in item_detail.find("h2").text:
            oceneni = item_detail.find_all(class_="col-md-3")
            oceneni = [cena.text.strip() for cena in oceneni]
            atributy["Ocenění"] = oceneni
            
        if "Hudba" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                atributy[div.find("h3").text] = [f"""{a.text} ({id_persony(a["href"])})""" for a in div.find_all("a")]
            
    item_details = soup.find_all(class_='film-description')
    
    for item_detail in item_details:
    
        if "Anotace" in item_detail.find("h2").text:
            atributy["Anotace"] = item_detail.find('p').text.strip()
            
        if "Obsah" in item_detail.find("h2").text:
            atributy["Obsah"] = item_detail.find('p').text.strip()
        
        if "Poznámka" in item_detail.find("h2").text:
            atributy["Poznámka"] = item_detail.find('p').text.strip()
        
    return(atributy)

## Test

In [19]:
scrape_film(slozka, "37478-patrani-po-ester.html")

Pátrání po Ester
37478


{'Film': 'Pátrání po Ester (37478)',
 'Země původu': 'Česká republika',
 'Copyright': '2005',
 'Rok výroby': '2005',
 'Premiéra': 'premiéra 2. 6. 2005 /přístupný/ (kino Světozor, Praha)',
 'Minutáž': '119 min',
 'Režie': 'Věra Chytilová (1555)',
 'Kategorie': 'film',
 'Žánr': 'životopisný, umělecký',
 'Typologie': 'dokumentární, distribuční',
 'Originální název': 'Pátrání po Ester',
 'Český název': 'Pátrání po Ester',
 'Anglický název': 'In Search of Ester',
 'Účinkují': ['Jaroslava Sacharová (22743)',
  'Michal Lázňovský (22745)',
  'Ivan Vyskočil (7030)',
  'Přemysl Rut (22746)',
  'Jiří Beran (22747)',
  'Boleslav Písařík (22748)',
  'Marie Uhlířová (22749)',
  'Olga Čechová (7316)',
  'Marie Jančová (22750)',
  'Edita Krumbachová (22751)',
  'Květa Fialová (5338)',
  'Kateřina Macháčková (11677)',
  'Dagmar Neumannová (22760)',
  'Jana Prachařová (8971)',
  'Drahomíra Hofmanová (22766)',
  'Jiřina Bohdalová (2878)',
  'Jan Schmid (12353)',
  'Boris Jachnin (22768)',
  'Ladislav Smo

In [20]:
test = scrape_film(slozka, "395570-snatkova-kancelar.html")
test

Sňatková kancelář
395570


{'Film': 'Sňatková kancelář (395570)',
 'Země původu': 'Československo',
 'Copyright': '1932',
 'Rok výroby': '1932',
 'Premiéra': 'premiéra 11. 3. 1932 /nepřístupný mládeži/ (kina Kotva /3 týdny/, Lucerna /3 týdny/ a Metro /2 týdny/, Praha)',
 'Minutáž': '95 min',
 'Režie': 'Svatopluk Innemann (33860)',
 'Kategorie': 'film',
 'Žánr': 'komedie',
 'Typologie': 'hraný, distribuční',
 'Originální název': 'Sňatková kancelář',
 'Český název': 'Sňatková kancelář',
 'Anglický název': 'Marriage Bureau',
 'Hrají': ['Jindřich Plachta (5829)',
  'Josef Král (130216)',
  'Zdeňka Baldová (28798)',
  'Jarmila Svatá (128701)',
  'Theodor Pištěk (37366)',
  'Milada Gampeová (122800)',
  'Ela Šárková (126039)',
  'Eman Fiala (64088)',
  'Jiřina Šejbalová (125559)',
  'Zvonimir Rogoz (127538)',
  'Růžena Šlemrová (127282)',
  'Václav Vydra ml. (126535)',
  'Hugo Kraus (100337)',
  'Jan W. Speerger (55928)',
  'J. O. Martin (51067)',
  'Josef Oliak (128024)',
  'Anna Švarcová (129670)',
  'Milka Balek-Br

## Todo

- víc odstavců anotace a obsahu

## Konečně DataFrame

In [21]:
import pandas as pd

In [22]:
kolik = None

In [23]:
def frejm(slozka):
    df = pd.DataFrame()
    total = len(seznam_souboru(slozka)[:kolik])
    count = 0
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"{count} / {total} / {s}")
        line = pd.DataFrame.from_dict([scrape_film(slozka, s)])
        df = pd.concat([df, line])
    df = df.reset_index(drop = True)
    return(df)

In [24]:
df = frejm("filmy")
df

1 / 4538 / 10144-tabor-padlych-zen.html
Tábor padlých žen
10144
2 / 4538 / 10173-eastern-krvavy-hugo.html
Eastern – Krvavý Hugo
10173
3 / 4538 / 102282-o-jecminkovi.html
O Ječmínkovi
102282
4 / 4538 / 102314-zimni-vila.html
Zimní víla
102314
5 / 4538 / 10242-narkoblues.html
Narkoblues
10242
6 / 4538 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
Jan Hus – mše za tři mrtvé muže
103341
7 / 4538 / 103354-obcansky-prukaz.html
Občanský průkaz
103354
8 / 4538 / 103358-7-dni-hrichu.html
7 dní hříchů
103358
9 / 4538 / 103360-vojta-lavicka-nahoru-a-dolu.html
Vojta Lavička: Nahoru a dolů
103360
10 / 4538 / 103369-kuky-se-vraci.html
Kuky se vrací
103369
11 / 4538 / 103372-habermannuv-mlyn.html
Habermannův mlýn
103372
12 / 4538 / 103374-dobre-placena-prochazka.html
Dobře placená procházka
103374
13 / 4538 / 103402-nebe-peklo.html
Nebe Peklo
103402
14 / 4538 / 103495-lidice.html
Lidice
103495
15 / 4538 / 103499-medvedi-ostrovy.html
Medvědí ostrovy
103499
16 / 4538 / 103509-dont-stop.html
DonT Stop
103

V peřině
159396
135 / 4538 / 15958-der-lebensborn-pramen-zivota.html
Der Lebensborn – Pramen života
15958
136 / 4538 / 15987-princezna-ze-mlejna-2.html
Princezna ze mlejna 2
15987
137 / 4538 / 1600-divoke-vcely.html
Divoké včely
1600
138 / 4538 / 16003-staroverci.html
Starověrci
16003
139 / 4538 / 160313-muj-vysvlecenej-denik.html
Můj vysvlečenej deník
160313
140 / 4538 / 160354-vrasky-z-lasky.html
Vrásky z lásky
160354
141 / 4538 / 160436-roman-pro-muze.html
Román pro muže
160436
142 / 4538 / 160465-hurvinek-na-scene-3d.html
Hurvínek na scéně 3D
160465
143 / 4538 / 162041-generace-60.html
Generace 60
162041
144 / 4538 / 162369-rock-zivota.html
Rock života
162369
145 / 4538 / 163239-tacho.html
Tacho
163239
146 / 4538 / 163291-je-treba-zabit-sekala.html
Je třeba zabít Sekala
163291
147 / 4538 / 16356-samotari.html
Samotáři
16356
148 / 4538 / 163656-kozi-pribeh-se-syrem.html
Kozí příběh se sýrem
163656
149 / 4538 / 163669-perfect-days.html
Perfect Days
163669
150 / 4538 / 163675-nevinnos

264 / 4538 / 199121-hra-o-kamen.html
Hra o kámen
199121
265 / 4538 / 199269-hleda-se-prezident.html
Hledá se prezident
199269
266 / 4538 / 199570-bastardi-iii.html
Bastardi III
199570
267 / 4538 / 19996-krysar.html
Krysař
19996
268 / 4538 / 200460-odpad-mesto-smrt.html
Odpad město smrt
200460
269 / 4538 / 20165-fricasse.html
Fricasse
20165
270 / 4538 / 201708-krizacek.html
Křižáček
201708
271 / 4538 / 201813-fulmaya-devcatko-s-tenkyma-nohama.html
Fulmaya, děvčátko s tenkýma nohama
201813
272 / 4538 / 201965-prepadeni.html
Přepadení
201965
273 / 4538 / 202283-zivot-s-kasparem.html
Život s Kašparem
202283
274 / 4538 / 202355-rok-bez-magora.html
Rok bez Magora
202355
275 / 4538 / 202450-zuzana-michnova-jsem-slavna-tak-akorat.html
Zuzana Michnová – Jsem slavná tak akorát
202450
276 / 4538 / 20261-narod-sobe-aneb-ceske-more-v-osmnacti-prilivech.html
Národ sobě aneb České moře v osmnácti přílivech
20261
277 / 4538 / 20268-nuda-v-brne.html
Nuda v Brně
20268
278 / 4538 / 203009-karaoke.html
Ka

402 / 4538 / 38577-postel.html
Postel
38577
403 / 4538 / 38585-panic-je-nanic.html
Panic je nanic
38585
404 / 4538 / 38587-raftaci.html
Rafťáci
38587
405 / 4538 / 38599-restart.html
Restart
38599
406 / 4538 / 38735-hezke-chvilky-bez-zaruky.html
Hezké chvilky bez záruky
38735
407 / 4538 / 38739-toyen.html
Toyen
38739
408 / 4538 / 38741-doom.html
Doom
38741
409 / 4538 / 38839-kletba-bratri-grimmu.html
Kletba bratří Grimmů
38839
410 / 4538 / 39263-indian-sestricka.html
Indián & sestřička
39263
411 / 4538 / 395-vyhnani-z-raje.html
Vyhnání z ráje
395
412 / 4538 / 395109-dostavenicko-ve-mlynici.html
Dostaveníčko ve mlýnici
395109
413 / 4538 / 395110-smich-a-plac.html
Smích a pláč
395110
414 / 4538 / 395111-vystavni-parkar-a-lepic-plakatu.html
Výstavní párkař a lepič plakátů
395111
415 / 4538 / 395112-to-nejlepsi-cislo.html
To nejlepší číslo
395112
416 / 4538 / 395113-jarni-sen-stareho-mladence.html
Jarní sen starého mládence
395113
417 / 4538 / 395114-hubicka.html
Hubička
395114
418 / 4538 /

533 / 4538 / 395230-za-cest-vitezu.html
Za čest vítězů
395230
534 / 4538 / 395231-za-svobodu-naroda.html
Za svobodu národa
395231
535 / 4538 / 395232-zlata-zena.html
Zlatá žena
395232
536 / 4538 / 395233-znamost-z-inseratu.html
Známost z inserátu
395233
537 / 4538 / 395234-zpev-zlata.html
Zpěv zlata
395234
538 / 4538 / 395235-setrele-pismo.html
Setřelé písmo
395235
539 / 4538 / 395236-babicka.html
Babička
395236
540 / 4538 / 395237-cerni-myslivci.html
Černí myslivci
395237
541 / 4538 / 395238-cesty-k-vysinam.html
Cesty k výšinám
395238
542 / 4538 / 395239-cikani.html
Cikáni
395239
543 / 4538 / 395240-dabluv-mlyn.html
Ďáblův mlýn
395240
544 / 4538 / 395241-deti-osudu.html
Děti osudu
395241
545 / 4538 / 395242-devcata-vdavejte-se.html
Děvčata, vdávejte se!
395242
546 / 4538 / 395243-devce-ze-stribrne-hranice.html
Děvče ze Stříbrné Hranice
395243
547 / 4538 / 395244-ferenc-se-zarizuje.html
Ferenc se zařizuje
395244
548 / 4538 / 395245-ircin-romanek-i.html
Irčin románek I.
395245
549 / 453

...

4032 / 4538 / 401985-humorista.html
Humorista
401985
4033 / 4538 / 401986-nekdy-je-to-jenom-reka.html
Někdy je to jenom řeka
401986
4034 / 4538 / 401987-clovek-kilimandzaro.html
Člověk Kilimandžáro
401987
4035 / 4538 / 401988-konzultace-posledni-den-zkouskoveho-obdobi.html
Konzultace poslední den zkouškového období
401988
4036 / 4538 / 401989-spolu-sami.html
Spolu sami
401989
4037 / 4538 / 401990-starici.html
Staříci
401990
4038 / 4538 / 401991-prvni-akcni-hrdina-a-jako-bonus-prvni-akcni-dokument.html
První akční hrdina a jako bonus: První akční dokument
401991
4039 / 4538 / 401992-jeste-nekoncime.html
Ještě nekončíme
401992
4040 / 4538 / 401993-narodni-trida.html
Národní třída
401993
4041 / 4538 / 401994-pan-ponrepo-se-klani.html
[Pan Ponrepo se klaní]
401994
4042 / 4538 / 401995-plac-svateho-sebestiana.html
Pláč svatého Šebestiána
401995
4043 / 4538 / 401996-abstinent.html
Abstinent
401996
4044 / 4538 / 401997-prvni-akcni-hrdina.html
První akční hrdina
401997
4045 / 4538 / 401998-prv

4161 / 4538 / 402123-jednotka-intenzivniho-zivota.html
Jednotka intenzivního života
402123
4162 / 4538 / 402124-manzelstvi.html
Manželství
402124
4163 / 4538 / 402125-laska-pod-kapotou.html
Láska pod kapotou
402125
4164 / 4538 / 402126-stingl-maly-velky-okima.html
Stingl – Malý velký Okima
402126
4165 / 4538 / 402127-poklad-v-balicku-nemecka-verze.html
Poklad v balíčku [německá verze]
402127
4166 / 4538 / 402128-kram-na-predmesti.html
Krám na předměstí
402128
4167 / 4538 / 402129-zajaty-hlas.html
Zajatý hlas
402129
4168 / 4538 / 402130-karel.html
Karel
402130
4169 / 4538 / 402131-gorbacov-raj.html
Gorbačov. Ráj
402131
4170 / 4538 / 402132-milan-kundera-od-zertu-k-bezvyznamnosti.html
Milan Kundera: Od Žertu k Bezvýznamnosti
402132
4171 / 4538 / 402133-namesicnici.html
Náměsíčníci
402133
4172 / 4538 / 402134-smrtici-zvuk.html
Smrtící zvuk
402134
4173 / 4538 / 402135-vecery-s-jindrichem-plachtou.html
Večery s Jindřichem Plachtou
402135
4174 / 4538 / 402136-muj-zivot-s-bohuslavem-martinu.h

4284 / 4538 / 48383-chytte-doktora.html
Chyťte doktora
48383
4285 / 4538 / 48718-marcela.html
Marcela
48718
4286 / 4538 / 48860-posledni-plavky.html
Poslední plavky
48860
4287 / 4538 / 48950-vetrelci-v-coloradu.html
Vetřelci v Coloradu
48950
4288 / 4538 / 495-cabriolet.html
Cabriolet
495
4289 / 4538 / 50479-jedne-noci-v-jednom-meste.html
Jedné noci v jednom městě
50479
4290 / 4538 / 50480-ulity.html
Ulity
50480
4291 / 4538 / 50482-jedne-noci-v-jednom-meste.html
Jedné noci v jednom městě
50482
4292 / 4538 / 50484-vetvicka-a-ploutvicka.html
Větvička a Ploutvička
50484
4293 / 4538 / 510-rebelove.html
Rebelové
510
4294 / 4538 / 51759-tobruk.html
Tobruk
51759
4295 / 4538 / 51760-medvidek.html
Medvídek
51760
4296 / 4538 / 51777-bobule.html
Bobule
51777
4297 / 4538 / 52494-protektor.html
Protektor
52494
4298 / 4538 / 52699-tajnosti.html
Tajnosti
52699
4299 / 4538 / 52734-ivetka-a-hora.html
Ivetka a hora
52734
4300 / 4538 / 52854-edith-piaf.html
Edith Piaf
52854
4301 / 4538 / 52996-praha-ocima

4411 / 4538 / 7396-cisarovy-nove-saty.html
Císařovy nové šaty
7396
4412 / 4538 / 7542-akumulator-1.html
Akumulátor 1
7542
4413 / 4538 / 7584-vekslak-aneb-stare-zlate-casy.html
Vekslák aneb Staré zlaté časy
7584
4414 / 4538 / 7585-vasnivy-polibek.html
Vášnivý polibek
7585
4415 / 4538 / 7586-krava.html
Kráva
7586
4416 / 4538 / 7587-hotylek-v-srdci-evropy.html
Hotýlek v srdci Evropy
7587
4417 / 4538 / 7596-ziletky.html
Žiletky
7596
4418 / 4538 / 77255-zapomenute-transporty-do-estonska.html
Zapomenuté transporty do Estonska
77255
4419 / 4538 / 77274-zapomenute-transporty-do-polska.html
Zapomenuté transporty do Polska
77274
4420 / 4538 / 7780-amerika.html
Amerika
7780
4421 / 4538 / 7782-jeste-vetsi-blbec-nez-jsme-doufali.html
Ještě větší blbec, než jsme doufali
7782
4422 / 4538 / 7783-lekce-faust.html
Lekce Faust
7783
4423 / 4538 / 7784-nexus.html
Nexus
7784
4424 / 4538 / 7785-princezna-ze-mlejna.html
Princezna ze mlejna
7785
4425 / 4538 / 7786-pevnost.html
Pevnost
7786
4426 / 4538 / 7787-r

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,cyklus
0,Tábor padlých žen (10144),"Slovensko, ...",1997,1997,premiéra 28. 9. 2000 /doporučená přístupnost o...,100 min,Laco Halama,film,"historický, tragikomedie","hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eastern – Krvavý Hugo (10173),Česká republika,1997,1997,nedistribuční premiéra 27. 4. 1997 ...,6 min,Aurel Klimt (1189),film,groteska,"animovaný, distribuční, studentský",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O Ječmínkovi (102282),Česká republika,2003,2002,nedistribuční premiéra 18. 4. 2003 (televizní...,61 min,Milan Cieslar (3593),film,pohádka,"hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Zimní víla (102314),Česká republika,1999,1999,nedistribuční premiéra 24. 12. 1999 (televizn...,52 min,Kryštof Hanzlík (1055),film,pohádka,"hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Narkoblues (10242),"Česká republika, ...",1997,1997,premiéra 10. 11. 2016 /přístupné bez omezení/,9 min,"Břetislav Pojar (3113), Ivan Vít (3030)",film,osvětový,"animovaný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4533,Hotel Sarajevo aneb Sny obyčejného svatebčana ...,Česká republika,1997,1997,premiéra 12. 8. 1997,NaN,Rodrigo Morales (3466),film,povídka,"hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4534,Zdivočelá země (9738),Česká republika,1997,1997,premiéra 13. 11. 1997 /přístupný/ (kino 64 U H...,114 min,Hynek Bočan (2947),film,"dobrodružný, historický","hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4535,Biograf (9769),Česká republika,1997,1997,nedistribuční premiéra 12. 6. 1997 ...,5 min,Galina Hůlová (7957),film,anekdota,"animovaný, distribuční, studentský",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4536,Knoflíkáři (9793),Česká republika,1997,1997,premiéra 20. 11. 1997 /doporučená přístupnost ...,102 min,Petr Zelenka (4806),film,"komedie, povídkový","hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df.columns

Index(['Film', 'Země původu', 'Copyright', 'Rok výroby', 'Premiéra', 'Minutáž',
       'Režie', 'Kategorie', 'Žánr', 'Typologie',
       ...
       'Výtvarník pozadí', 'Výrobní skupina', 'Zákazník', 'Vývozní společnost',
       'Umělecké vedení', 'Zpěv role', 'Překlad české verze', 'Rychlost',
       'Přístupnost', 'cyklus'],
      dtype='object', length=130)

## Export

In [26]:
df.to_json(os.path.join("data", "filmy_raw.json"), orient="records")